In [29]:
pip install textblob 

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# BM


In [1]:
import pandas as pd
from textblob import TextBlob
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Carica il dataset
df = pd.read_excel("ConversationDataset.xlsx")

# Download delle risorse necessarie
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Pre-processing del testo
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Rimozione della punteggiatura
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    
    # Tokenizzazione
    tokens = nltk.word_tokenize(text)
    
    # Rimozione delle stop words e lemmatizzazione delle parole
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]
    
    # Ricomposizione del testo preprocessato
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

# Funzione per l'analisi del sentiment utilizzando TextBlob
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    
    # Assegna il sentiment in base al valore di polarità
    if sentiment > 0:
        sentiment_label = "Negativo"
    elif sentiment <= 0:
        sentiment_label = "Positivo"
    return sentiment_label

# Aggiungi una colonna per il sentiment analysis
df['Sentiment'] = df['Text'].apply(analyze_sentiment)

# Conta il numero totale di righe
total = len(df)

# Conta quanti tweet per ciascun sentimento
counts = df['Sentiment'].value_counts()

# Estrai i valori, default 0 se non presenti
neg = counts.get('Negativo', 0)
neu = counts.get('Neutrale', 0)
pos = counts.get('Positivo', 0)

# Calcola le percentuali
neg_pct = (neg / total) * 100
neu_pct = (neu / total) * 100
pos_pct = (pos / total) * 100

# Stampa i risultati
print(f"Negative: {neg_pct:.2f}%")
print(f"Neutral: {neu_pct:.2f}%")
print(f"Positive: {pos_pct:.2f}%")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Negative: 30.21%
Neutral: 0.00%
Positive: 69.79%


# Vader


In [2]:
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import nltk

# Assicurati di avere il lexicon
nltk.download('vader_lexicon')

# Carica il dataset
df = pd.read_excel("ConversationDataset.xlsx")

# Inizializza VADER
sia = SentimentIntensityAnalyzer()

def classify_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.4:
        return 'Positive'
    elif score <= -0.05:  # più permissiva verso il negativo
        return 'Neutral'
    else:
        return 'Negative'



# Applica la classificazione
df['Sentiment'] = df['Text'].apply(classify_sentiment)

# Calcola le percentuali
sentiment_counts = df['Sentiment'].value_counts(normalize=True) * 100
sentiment_percentages = sentiment_counts.round(2)

# Mostra i risultati
print(sentiment_percentages)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment
Negative    44.47
Neutral     38.16
Positive    17.37
Name: proportion, dtype: float64


# Vader + NLTK


In [3]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download risorse NLTK (solo la prima volta)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

# Carica il dataset
df = pd.read_excel("ConversationDataset.xlsx")

# Preprocessing del testo
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Rimuovi punteggiatura
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    # Tokenizza
    tokens = nltk.word_tokenize(text)
    # Lemmatizza e rimuovi stop words
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]
    return ' '.join(tokens)

df['preprocessed_text'] = df['Text'].apply(preprocess_text)

# Inizializza VADER
sia = SentimentIntensityAnalyzer()

# Funzione di classificazione con soglie personalizzate
def classify_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.4:
        return 'Positive'
    elif score <= -0.05:
        return 'Neutral'
    else:
        return 'Negative'

# Applica la classificazione al testo preprocessato
df['Sentiment'] = df['preprocessed_text'].apply(classify_sentiment)

# Calcola percentuali
percentages = df['Sentiment'].value_counts(normalize=True) * 100
percentages = percentages.round(2)
print("Distribuzione del sentiment (%):")
print(percentages)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/claudiasantoro/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Distribuzione del sentiment (%):
Sentiment
Negative    45.10
Neutral     37.39
Positive    17.51
Name: proportion, dtype: float64


# Laser


In [84]:
pip install torch==1.9.1


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Obtaining dependency information for torch==1.9.1 from https://files.pythonhosted.org/packages/73/f1/835fad3d4c47671debeb71e798b8d32d84b50a66b58b95973daee2d62929/torch-1.9.1-cp39-none-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 MB 3.4 MB/s eta 0:00:0000:01m0:01m
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Confi

In [87]:
!pip3 install tensorflow
!pip3 install transformers[sentencepiece]


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 259.5 MB 22 kB/s s eta 0:00:01     |███████████████▌                | 125.7 MB 7.8 MB/s eta 0:00:18
     |████████████████████████████████| 5.5 MB 31.5 MB/s eta 0:00:01
     |████████████████████████████████| 394 kB 32.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 37.2 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 599 kB/s  eta 0:00:01
     |████████████████████████████████| 3.4 MB 37.9 MB/s eta 0:00:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |████████████████████████████████| 11.3 MB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 30.0 MB/s eta 0:00:01
     |████████████████████████████████| 389 kB 24.7 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 26.5 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 

In [ ]:
from laserembeddings import Laser
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
from laserembeddings import Laser
from transformers import pipeline

# Load the LASER language model for text embeddings
laser = Laser()

# Load the sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis")

# Read your dataset
df = pd.read_excel("ConversationDataset.xlsx")

# Define a function to preprocess text and perform sentiment analysis
def analyze_sentiment_with_laser(text):
    # Preprocess text (you can modify this as needed)
    text = text.lower()
    
    # Generate text embeddings using LASER
    embeddings = laser.embed_sentences([text], lang='en')
    
    # Perform sentiment analysis using the sentiment analyzer
    sentiment_result = sentiment_analyzer(text)[0]
    
    # Extract sentiment label and score
    sentiment_label = sentiment_result['label']
    sentiment_score = sentiment_result['score']
    
    return sentiment_label, sentiment_score

# Apply sentiment analysis to each row in the DataFrame
sentiment_results = df['Text'].apply(analyze_sentiment_with_laser)

# Extract sentiment labels and scores into separate columns
df['Sentiment_Label'] = [result[0] for result in sentiment_results]
df['Sentiment_Score'] = [result[1] for result in sentiment_results]

# Save the DataFrame with sentiment analysis results
df.to_excel("Sentiment_Analysis_Laser.xlsx", index=False)


In [107]:
import pandas as pd

# Carica il file Excel
df = pd.read_excel("Sentiment_Analysis_Laser.xlsx")  # <-- Cambia il nome per ogni file

# Conta il numero totale di righe
total = len(df)

# Conta quanti tweet per ciascun sentimento
counts = df['Sentiment_Label'].value_counts()

# Estrai i valori, default 0 se non presenti
neg = counts.get('NEGATIVE', 0)
pos = counts.get('POSITIVE', 0)
neu = counts.get('NEUTRAL', 0)

# Calcola le percentuali
neg_pct = (neg / total) * 100
neu_pct = (neu / total) * 100
pos_pct = (pos / total) * 100

# Stampa i risultati
print(f"Negative: {neg_pct:.2f}%")
print(f"Neutral: {neu_pct:.2f}%")
print(f"Positive: {pos_pct:.2f}%")


Negative: 82.63%
Neutral: 0.00%
Positive: 17.37%
